![](image_text/120a_02.png)

![](image_text/120a_03.png)

![](image_text/120a_04.png)

![](image_text/120a_05.png)

![](image_text/120a_06.png)

![](image_text/120a_07.png)

![](image_text/120a_08.png)

![](image_text/120a_09.png)

![](image_text/120a_10.png)

以下で、左に表示される入出力セル番号は新規実行した場合の番号を示します。

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Lasso
import matplotlib.pyplot as plt
import os

In [ ]:
DATA_NAME = "L64"  #　フォントを定義
# ”L64", "dance64", "syou64", "zen64"を選択可能です。
D = 4 # トモグラフ像撮影数を説明変数の1/Dとする。
NOISE_FAC = 0.05  # Xwに追加するノイズの大きさ
ALPHA = 10**(-5)  # 予め最適化したLasso hyperparemeter
FIT_INTERCEPT = True # Lassoの切片を使うかどうか。


In [ ]:
ROOT = ".."
filename = os.path.join(f"{ROOT}/data/font",DATA_NAME + ".csv") # ファイル名生成
data = np.loadtxt(filename, delimiter=",") # 名前カラムがないので、numpyでデータ取得している。
l = data.shape[0] # data(l,l)、フォント。(l,l)であることを仮定している。
print("l=",l)
data = data.reshape(-1) # P=l*l, size Pの一次元に行列を変える。


In [ ]:
m1 = data.min() # 全体の最小値
m2 = data.max() # 全体の最大値
w_orig = (data-m1)/(m2-m1) # データ全体を[最小値,最大値]を[0,1]で規格化する。
# 元画像をw_origとする。地（背景）部分が0、文字部分が1となる。


In [ ]:
plt.imshow(w_orig.reshape(l,l), cmap=plt.cm.gray, interpolation='nearest') # データの可視化

In [ ]:
plt.hist(w_orig, bins=100) # 0が多いことを仮定していたので、値の頻度を可視化
"_" # plt.histだけだとtext出力が多く出るのでcode cellの最後に文字列を表示
# Lの場合82％が=0。

In [ ]:
from tomography_misc import build_projection_operator
# build_projection_operatorはsklearnのコードから来ています．
# https://scikit-learn.org/stable/auto_examples/applications/plot_tomography_l1_reconstruction.html
P = l*l # 説明変数数
X = build_projection_operator(l, l//D) # 変換行列Xの生成, l//Dはintを返す。Nをl//D分の１にする。
print(X.shape) # (N,P), D=4だとPの1/4なのでN=4069/4=1024になる。

In [ ]:
y = X * w_orig # 平行系トモグラフ像の生成, X * w　
y += NOISE_FAC * (np.random.randn(*y.shape)-0.5)*2 # 更に乱数ノイズを加えて観測値とする。


In [ ]:
reg = Lasso(alpha=ALPHA, fit_intercept=FIT_INTERCEPT) # 予め設定したALPHAを用いてLasso
reg.fit(X, y) # Xとyからwを求める＝トモグラフ像の復元
w_reg = reg.coef_ # w_reg(P) 線型回帰の係数(.coef_)が復元された像,w_regとしておく。


In [ ]:
from tomography_misc import plot_images
plot_images(w_orig.reshape(l,l), w_reg.reshape(l,l), "original", "L1") 
# original（左）とLassoの結果（右）を並べて表示する。
# image（上）と値の頻度（下）

In [ ]:
# 最小を0、最大を1とする。
w_reg_max1 = np.fmin(w_reg, 1) 
w_reg_max1 = np.fmax(w_reg_max1, 0) 

plot_images(w_orig.reshape(l,l), w_reg_max1.reshape(l,l), "original", "L1 [0:1]") 
# originalとLassoの結果を並べて表示する。
# image（上）と値の頻度（下）

![](image_text/120a_11.png)

## 文字ソース
- 文字Lは https://commons.wikimedia.org/wiki/File:Old_English_typeface.svg
- 日本語フォントは「筆文字フリー素材集」http://fudemoji-free.com/

から取得しました。